In [1]:
import numpy as np
import pandas as pd
import re
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from scipy.stats import norm, lognorm, anderson, kstest
from sklearn import metrics
from rf_functions import data_setup, run_rf
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
pd.options.mode.chained_assignment = None

In [13]:
s1_fp = 'input_data/s1_zonal_data.csv'
stats_fp = 'output/stats/scale_shape_both_dates.csv'
moran_fp = 'input_data/moran_max.csv'
output_d = 'output/'
out_file_prefix = 'func_test_dB'

targets, predictors = data_setup(s1_fp, stats_fp, moran_fp, s1_units='dB', date=None)
cols_0219 = [c for c in targets.columns if '0219' in c]
cols_0302 = [c for c in targets.columns if '0304' in c]
cols_misc = [c for c in targets.columns if c not in targets_0219.columns and c not in targets_0302.columns]
cols_misc.remove('Point_ID')
cols_misc.remove('moran_p')
cols_0219 += cols_misc
cols_0302 += cols_misc
targets_0219 = targets[cols_0219].dropna()
targets_0302 = targets[cols_0302].dropna()

run_rf(targets_0219, predictors.loc[targets_0219.index], n_runs=100, 
       rf_type='single_target', output_dir=output_d, out_file_prefix=out_file_prefix)
run_rf(targets_0302, predictors.loc[targets_0302.index], n_runs=100, 
       rf_type='single_target', output_dir=output_d, out_file_prefix=out_file_prefix)


NameError: name 'dfdf' is not defined

In [18]:
targets_0219.head()

zonal_0219_5m_p95_max  zonal_0219_5m_p95_min  \
date S1_pixel_ID                                                 
0218 1                         0.021105               0.010579   
     2                         0.008444               0.005658   
     3                         0.006577               0.004523   
     4                         0.009890               0.005861   
     5                         0.010825               0.003736   

                  zonal_0219_5m_p95_mean  zonal_0219_5m_p5_max  \
date S1_pixel_ID                                                 
0218 1                          0.017823              0.001797   
     2                          0.007096              0.001009   
     3                          0.005532              0.000882   
     4                          0.007330              0.000838   
     5                          0.008685              0.001239   

                  zonal_0219_5m_p5_min  zonal_0219_5m_p5_mean  \
date S1_pixel_ID                                                
0218 1                        0.001096               0.001330   
     2                        0.000876               0.000946   
     3                        0.000739               0.000810   
     4                        0.000795               0.000811   
     5                        0.000604               0.000927   

                  zonal_0219_5m_max_min  zonal_0219_5m_max_mean  \
date S1_pixel_ID                                                  
0218 1                         0.057575                0.079862   
     2                         0.019491                0.025016   
     3                         0.014456                0.015568   
     4                         0.019198                0.039431   
     5                         0.014270                0.068614   

                  zonal_0219_5m_med_max  zonal_0219_5m_med_min  ...  \
date S1_pixel_ID                                                ...   
0218 1                         0.006456               0.003473  ...   
     2                         0.002946               0.002325  ...   
     3                         0.002376               0.001869  ...   
     4                         0.002299               0.002120  ...   
     5                         0.003750               0.001432  ...   

                  roughness_p25  roughness_p75  roughness_p95       iqr  \
date S1_pixel_ID                                                          
0218 1                 0.002675       0.008086       0.018360  0.005411   
     2                 0.001725       0.004034       0.007187  0.002309   
     3                 0.001428       0.003210       0.005620  0.001782   
     4                 0.001450       0.003412       0.007025  0.001961   
     5                 0.001558       0.004713       0.009320  0.003155   

                    p95-p5     scale     shape        ks     moran    moran_z  
date S1_pixel_ID                                                               
0218 1            0.017106  0.004689  0.816457  0.006510  0.595374  33.320187  
     2            0.006250  0.002629  0.619090  0.010317  0.607223  33.988191  
     3            0.004819  0.002132  0.591836  0.006076  0.577542  32.252885  
     4            0.006214  0.002266  0.660623  0.021772  0.740216  41.615541  
     5            0.008532  0.002731  0.776050  0.019981  0.739956  42.180551  

[5 rows x 91 columns]

In [ ]:
[x for x in targets_0219.index if x[0] == targets_0219.index.levels[0][1]]
targets_0219.index[0][0]